# Factorización matricial con Keras

**Autor**: Arturo Sánchez Palacio

Basado en: https://github.com/lazyprogrammer

**Fecha de última revisión: 17/I/2020**

Como siempre comenzamos instalando los módulos que necesitamos e importándolos:

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

Keras está optimizado para trabajar con enormes tamaños de datos por lo que en este caso trabajaremos con la base de datos __completa__:

In [2]:
df = pd.read_csv('./data/preprocessed_rating.csv')

Calculamos el número total de usuarios y películas:

In [3]:
N = df.userId.max() + 1
M = df.movie_idx.max() + 1

Como es habitual en los procesos de Machine Learning dividimos en entrenamiento y test. En este caso fijamos el tamaño de entrenamiento al 80%:

In [4]:
df = shuffle(df)
umbral = int(0.8*len(df))
df_train = df.iloc[:umbral]
df_test = df.iloc[umbral:]

Inicializamos las variables habituales:

In [5]:
K = 10 # dimension latente
mu = df_train.rating.mean() #media global
iteraciones = 15 
reg = 0. # coeficiente de regularización: ajuste de parámetros

Ya hemos fijado las bases para construir el modelo. Importamos los tipos de capas que vamos a usar:

In [6]:
from keras.layers import Input, Embedding, Dot, Add, Flatten

Using TensorFlow backend.


Empezamos creando las capas de entrada:

In [7]:
u = Input(shape=(1,)) # escalar del usuario
m = Input(shape=(1,)) # escalar de la película

Realizamos el embedding con la regularización del vector de usuario y de película en las matrices:

In [8]:
from keras.regularizers import l2
u_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(u) # dimensión: (N, 1, K)
m_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(m) # dimensión: (N, 1, K)

Una vez introducidos la película y el usuario introducimos el sesgo:

In [9]:
u_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(u) # dimensión: (N, 1, 1)
m_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(m) # dimensión: (N, 1, 1)

Calculamos el producto del embedding de los usuarios y las películas:

In [10]:
x = Dot(axes=2)([u_embedding, m_embedding]) # (N, 1, 1)
#Aquí no hay aprediaje, solo un producto

Sumamos al resultado anterior los sesgos:

In [11]:
x = Add()([x, u_bias, m_bias])

Usamos una capa `Flatten` para garantizar que la salida es 1 x 1. Recordemos que buscamos el score de la recomendación:

In [12]:
x = Flatten()(x) # (N, 1)

Esta es la estructura final del modelo. Instanciamos el modelo:

In [13]:
from keras.models import Model
model = Model(inputs=[u, m], outputs=x)

Compilamos el modelo:

In [14]:
from keras.optimizers import SGD, Adam
model.compile(
  loss='mse',#usamos como función de pérdida el error medio cuadrático
  optimizer=SGD(lr=0.08, momentum=0.9), #usamos como optimizador SGD con learning rate 0.08 y momentum 0.9
  metrics=['mse'], #empleamos como métric a el error medio cuadrático
)

Este es el modelo construido:

In [15]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 10)        1384930     input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 10)        267440      input_2[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

Nuestro modelo ya está listo para ser entrenado:

In [ ]:
#A partir solamente de los identificadores se obtienen los ratings.
r = model.fit(
  x=[df_train.userId.values, df_train.movie_idx.values], #variables de entrada
  y=df_train.rating.values - mu, #etiquetas a predecir
  epochs=iteraciones,
  batch_size=128, #tamaño del batch
  validation_data=([df_test.userId.values, df_test.movie_idx.values],  df_test.rating.values - mu) #datos de validacion
)



Train on 16000210 samples, validate on 4000053 samples
Epoch 1/15
 4842240/16000210 [========>.....................] - ETA: 11:36 - loss: 0.8317 - mean_squared_error: 0.8317

El objeto r almacena en su historia las pérdidas a lo largo del entrenamiento y podemos tanto observarlas como representarlas:

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="Pérdida entrenamiento")
plt.plot(r.history['val_loss'], label="Pérdida validación")
plt.legend()
plt.show()

# plot mse
plt.plot(r.history['mse'], label="Error medio cuadrático (entrenamiento)")
plt.plot(r.history['val_mse'], label="Erros medio cuadrático (test)")
plt.legend()
plt.show()

## Conclusión

La parte más interesante de este modelo es que al ser mucho más rápido nos permite entrenar con todos los datos en lugar de la muestra. La manera de construir el modelo además es mucho más sencilla y escalable. En cuanto a resultados podemos observar que las pérdidas con una red bastante sencilla son asimilables a lo visto hasta ahora.